## Data loading

미리 만들어둔 document - term matrix 를 이용하여 LDA 를 학습합니다. X 는 (document, term) matrix 입니다.

In [1]:
import pickle
import config
from lovit_textmining_dataset.navernews_10days import get_bow

TRAIN_LDA = True

x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')
x.shape

soynlp=0.0.49
added lovit_textmining_dataset


(30091, 9774)

## Version check

Gensim 은 scipy 를 이용합니다. 이전 버전의 scipy 를 이용하는 경우 학습 시 오류가 발생할 수 있습니다. scipy 의 버전을 확인합니다.

In [2]:
import gensim
import scipy

print('gensim == {}'.format(gensim.__version__))
print('scipy == {}'.format(scipy.__version__))

gensim == 3.6.0
scipy == 1.1.0


## Train data format

Gensim 의 LDA 가 이용하는 학습데이터의 형식 각 문서를 list of tuple 로 표현하는 Corpus 라는 형식입니다. tuple 은 (term_index, frequency) 입니다.

```
from gensim import corpora

# create a toy corpus of 2 documents, as a plain Python list
corpus = [[(1, 0.5)], []]
```

scikit-learn 을 이용하여 sparse matrix 를 미리 만들었다면 gensim.matutils.Sparse2Corpus 를 이용하여 이를 변형합니다. 그런데, gensim 은 sparse matrix 를(term, doc) matrix 라고 가정합니다. 그래서 Sparse2Corpus 의 documents_columns 를 False 로 지정해줍니다. Default 는 True 입니다. Corpus format 은 [여기][corpus_format]를 참고하세요

```
from scipy.io import mmread

scipy_sparse_matrix = mmread(mm_fname)    
# corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
corpus = gensim.matutils.Sparse2Corpus(x, documents_columns=False)
```


[corpus_format]: https://radimrehurek.com/gensim/tut1.html#corpus-formats

In [3]:
corpus = gensim.matutils.Sparse2Corpus(x, documents_columns=False)

for i, doc in enumerate(corpus):
    if i > 3: break
    print('#doc= %d: ' % i, doc[:10], '...')

#doc= 0:  [] ...
#doc= 1:  [(200, 1), (219, 1), (593, 1), (349, 1)] ...
#doc= 2:  [(6161, 1), (8337, 2), (6274, 7), (1104, 3), (4627, 7), (6017, 5), (1244, 8), (1499, 1), (234, 4), (6170, 9)] ...
#doc= 3:  [(6017, 2), (234, 1), (5668, 1), (9051, 2), (3732, 1), (8883, 1), (6738, 1), (5179, 1), (1997, 1), (4797, 1)] ...


30,091 개의 문서임을 확인할 수 있습니다.

In [4]:
len(corpus)

30091

Gensim LDA 는 각 column index 에 해당하는 단어를 list of str 이 아닌 {int:str} 형식으로 저장합니다.

In [5]:
id2word = dict(enumerate(idx_to_vocab))
id2word[5537]

'아이오아이'

## Train LDA

이제 데이터를 gensim LDA format으로 맞추는 일이 끝났습니다.

num_topics 는 토픽의 개수를 정하는 부분입니다. id2word 를 입력하지 않으면 단어가 term index 로 출력됩니다. Gensim 의 id2word 는 list of str 이 아닌, dict 구조여야 합니다.

In [6]:
%%time

from gensim.models import LdaModel

ldamodel_fname = './2016-10-20-lda.pkl'

if TRAIN_LDA:
    ldamodel = LdaModel(corpus=corpus, num_topics=100, id2word=id2word)
    import pickle
    with open(ldamodel_fname, 'wb') as f:
        pickle.dump(ldamodel, f)
else:
    with open(ldamodel_fname, 'rb') as f:
        ldamodel = pickle.load(f)

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


CPU times: user 2min 19s, sys: 6min 24s, total: 8min 43s
Wall time: 49.9 s


print_topic은 특정 topic에 대하여 설명력이 좋은 (topic probability가 높은) topn개의 단어를 prob.와 함께 출력해줍니다. 

In [7]:
ldamodel.print_topic(0, topn=5)

'0.103*"브랜드" + 0.031*"6개" + 0.025*"공간" + 0.025*"시즌" + 0.023*"냉장고"'

각 topic 의 단어생성확률은 아래의 함수로 가져올 수 있습니다. LdaModel.expElogbeta 에도 비슷한 정보가 포함되어 있지만, 이는 학습에 이용하는 temporal variable 입니다.

In [8]:
ldamodel.expElogbeta.shape

(100, 9774)

정확한 topic - term probability 는 아래의 함수를 이용해야 합니다.

In [9]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

beta = get_topic_term_prob(ldamodel)
beta.shape

(100, 9774)

각 topic 별로 생성 확률이 큰 10 개의 단어들을 선택합니다.

In [10]:
topn = 10
important_terms = beta.argsort(axis=1)[:,-topn:]
print(important_terms.shape)

(100, 10)


위의 print_topic() 함수의 결과는 아래의 과정을 거쳐 출력된 함수입니다.

In [11]:
for topic_idx in range(beta.shape[0]):
    if topic_idx % 5 == 0:
        print()

    term_indices = important_terms[topic_idx,:].reshape(-1)
    terms = reversed([idx_to_vocab[idx] for idx in term_indices])

    print('topic #{} : {}'.format(topic_idx, ' '.join(terms)))


topic #0 : 브랜드 6개 공간 시즌 냉장고 출시 보관 신제품 마케팅 이지
topic #1 : 공개 출연 영화 사진 매력 선보 앨범 기자 함께 팬들
topic #2 : 감독 주지홍 제작진 김윤혜 친구 기자 첫사랑 세상 모습 종영
topic #3 : 훈련 기뢰 가계대출 개편 기대감 218 해군 실전 계열사인 레드
topic #4 : 중국 미국 대통령 주석 베이징 양국 협력 분쟁 남중국해 기증

topic #5 : 원전 운영 건설 세계 이번 수출 수주 체결 사업 한국형
topic #6 : 고문 사장 재판 1심 상임고문 관할 판결 법원 사람 이혼
topic #7 : 회의 의학 참석 장관 인사 20일 기자 나누고 이승 16일
topic #8 : 울산 방지 진화 이동 디지털 충돌 장착 대우조선 칼럼니스트 조사
topic #9 : 총기 경찰 사제 성씨 서울 경위 제작 총격 인근 사제총

topic #10 : 제품 라인 사용 부장 전자 생산 스마트폰 재배포 교체 세트장
topic #11 : 지원 협약 체결 경기 확대 농산물 농촌 사회공헌 합격자 상생
topic #12 : 방송 시청자들 뉴미디어 시청자 정원 질문 깜짝 보도자료 시사 상대방
topic #13 : 트럼프 클린턴 토론 미국 후보 대선 3차 앵커 대통령 주장
topic #14 : 트와이스 08 영주 평가 아시아 사회적 지난주 지수 기업 올해

topic #15 : 재배포 두테르테 작품 작가 수상 정상회담 드라마 롤링 잠시 11시
topic #16 : 네이버 여성 독도 서비스 검색 인터넷 이탈 포털 공무원들 현실
topic #17 : 전시 공연 함께 제공 축제 관광 관광객 진행 발굴 이번
topic #18 : 서비스 고객 은행 가입 이용 상품 보험 카드 금액 중복
topic #19 : 승객 열차 출입문 출발 운전 목숨 어제 목격자 신고 속보

topic #20 : 사고 기관사 전동차 확인 조사 안전문 김포공항역 오전 발생 경찰
topic #21 : 오후 전남 오전 21일 22일 시민들 시민 19일 30분 발인

gensim 에서 단어의 topic vector 를 직접적으로 찾아주는 함수가 구현되어 있지 않습니다. 하지만 LdaModel[bow_model]을 넣으면 bow_model 에 대한 topic vector 를 출력해줍니다. bow_model 에 단어 한 개를 넣으면 해당 단어의 topic vector 를 알 수 있습니다. 

bow_model 은 [(term id, weight), (term id, weight), ... ] 형식입니다. 

In [12]:
ldamodel[[(0,1)]]

[(53, 0.50500005)]

In [13]:
def encode(words):
    bow = [(vocab_to_idx.get(w, -1), f) for w,f in words]
    bow = [(w, t) for w, t in bow if w != -1]
    return bow

In [14]:
bow_model = encode([('트와이스', 1)])
ldamodel[bow_model]

[(14, 0.505)]

get_document_topics() 함수를 이용하면 minimum_probability, minimum_phi_value를 조절하며 topic vector를 만들 수 있습니다. `트와이스`는 14 번 topic 에서 자주 생성되는 단어입니다. 하지만 topic weight 가 50.5 % 밖에 되지 않습니다. 이는 Gensim LDA 가 다른 topic 에 기본확률로 0.01 을 할당하고 있기 때문엡니다.

In [15]:
bow_model = encode([('트와이스', 1)])
ldamodel.get_document_topics(bow_model,  minimum_probability=0.01, minimum_phi_value=0.01)

[(14, 0.50500005)]

`무대`라는 단어는 25 번 topic 에 크게 할당되어 있습니다. 하지만 이 역시 50.5 % 만 할당되어 있습니다. `minimum_probability` 를 0.005 로 설정하면 다른 topic 의 weight 도 출력이 됩니다.

In [21]:
bow_model = encode([('무대', 1)])
ldamodel.get_document_topics(bow_model,  minimum_probability=0.005, minimum_phi_value=0.005)

[(0, 0.0050000004),
 (1, 0.0050000004),
 (2, 0.0050000004),
 (3, 0.0050000004),
 (4, 0.0050000004),
 (5, 0.0050000004),
 (6, 0.0050000004),
 (7, 0.0050000004),
 (8, 0.0050000004),
 (9, 0.0050000004),
 (10, 0.0050000004),
 (11, 0.0050000004),
 (12, 0.0050000004),
 (13, 0.0050000004),
 (14, 0.0050000004),
 (15, 0.0050000004),
 (16, 0.0050000004),
 (17, 0.0050000004),
 (18, 0.0050000004),
 (19, 0.0050000004),
 (20, 0.0050000004),
 (21, 0.0050000004),
 (22, 0.0050000004),
 (23, 0.0050000004),
 (24, 0.0050000004),
 (25, 0.505),
 (26, 0.0050000004),
 (27, 0.0050000004),
 (28, 0.0050000004),
 (29, 0.0050000004),
 (30, 0.0050000004),
 (31, 0.0050000004),
 (32, 0.0050000004),
 (33, 0.0050000004),
 (34, 0.0050000004),
 (35, 0.0050000004),
 (36, 0.0050000004),
 (37, 0.0050000004),
 (38, 0.0050000004),
 (39, 0.0050000004),
 (40, 0.0050000004),
 (41, 0.0050000004),
 (42, 0.0050000004),
 (43, 0.0050000004),
 (44, 0.0050000004),
 (45, 0.0050000004),
 (46, 0.0050000004),
 (47, 0.0050000004),
 (48, 0.0

새로운 문서에 대해서도 BOW list 를 만들어 topic probability vector 를 inferring 할 수 있습니다.

In [22]:
bow_model = encode([('트와이스', 2), ('무대', 1)])
ldamodel.get_document_topics(bow_model, minimum_probability=0.01, minimum_phi_value=0.005)

[(14, 0.5025402), (25, 0.25245988)]